In [0]:
#Type1
current_dim_df = spark.read.csv("current_dimension_data.csv", header=True)
new_dim_df = spark.read.csv("new_dimension_data.csv", header=True)

In [0]:
current_dim_df.createOrReplaceTempView("current_dim")
new_dim_df.createOrReplaceTempView("new_dim")

In [0]:
updated_dim_df = spark.sql("""
    SELECT
        new_dim.id AS id,
        new_dim.name AS name,
        new_dim.description AS description
    FROM
        new_dim
""")

In [0]:
updated_dim_df.write.mode("overwrite").csv("current_dimension_data.csv")

In [0]:
#Type2
current_dim_df = spark.read.csv("current_dimension_data.csv", header=True)
new_dim_df = spark.read.csv("new_dimension_data.csv", header=True)

In [0]:
# Add a start and end date to the current and new dimension data
current_dim_df = current_dim_df.withColumn("end_date", current_date())
new_dim_df = new_dim_df.withColumn("start_date", current_date())

In [0]:
# Identify rows in the current dimension that are updated
updated_rows = current_dim_df.join(new_dim_df, on=["id"], how="inner")

In [0]:
# Mark the old rows as expired
updated_rows = updated_rows.withColumn("end_date", updated_rows.start_date)
updated_dim_df = updated_rows.union(new_dim_df)

In [0]:
updated_dim_df.write.mode("overwrite").csv("current_dimension_data.csv")

In [0]:
# Load the current dimension data from a source
current_dim_df = spark.read.csv("current_dimension_data.csv", header=True)

# Load the new dimension data from another source
new_dim_df = spark.read.csv("new_dimension_data.csv", header=True)

# Update the current dimension data with new data, keeping track of previous values
current_dim_df.createOrReplaceTempView("current_dim")
new_dim_df.createOrReplaceTempView("new_dim")

updated_dim_df = spark.sql("""
    SELECT
        current_dim.id AS id,
        new_dim.name AS current_name,
        current_dim.current_description AS previous_description,
        new_dim.description AS current_description
    FROM
        current_dim
    JOIN
        new_dim
    ON
        current_dim.id = new_dim.id
""")
updated_dim_df.write.mode("overwrite").csv("current_dimension_data.csv")

spark.stop()
